In [19]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
from IPython import display
import datetime
import time

import glob
import os
import cv2
import tqdm
from matplotlib.colors import LinearSegmentedColormap
from tqdm.notebook import trange, tqdm

from utils import *
from deep_shadow import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# % bos 19832 24240 summer
# % bue 22127 39476 spring
# % dc 18730 25074 spring
# % joh 37872 37714 winter
# % joh 37873 37713 spring
# % sea 10496 22885 winter
# % sea 10497 22885 winter
# % sea 10497 22887 summer
# % tok 58211 25805 winter

colors = [(1, 1, 1), (254/255,204/255,92/255), (253/255,141/255,60/255), (240/255,59/255,32/255), (189/255,0,38/255)]
_cmap = LinearSegmentedColormap.from_list('colorscale', colors, N=100)

# if not exists create folders like ablation_study/bos_19832_24240_summer
ablation = [['bos', 19832, 24240, 'summer'], ['bue', 22127, 39476, 'spring'], ['dc', 18730, 25074, 'spring'], ['joh', 37872, 37714, 'winter'], 
            ['joh', 37873, 37713, 'spring'], ['sea', 10496, 22885, 'winter'], ['sea', 10497, 22885, 'winter'], ['sea', 10497, 22887, 'summer'], ['tok', 58211, 25805, 'winter']]

# with tqdm
# for i in range(len(ablation)):
for i in tqdm(range(len(ablation))):
    if not os.path.exists('data/ablation_study/%s_%d_%d_%s/'%(ablation[i][0], ablation[i][1], ablation[i][2], ablation[i][3])):
        os.makedirs('data/ablation_study/%s_%d_%d_%s/'%(ablation[i][0], ablation[i][1], ablation[i][2], ablation[i][3]))

    img = cv2.imread("data\\shadows\\%s-%s\\16\\%d\\%d.png"%(ablation[i][0], ablation[i][3], ablation[i][1], ablation[i][2]), cv2.IMREAD_GRAYSCALE)
    height = cv2.imread("data\\heights_new\\%s\\16\\%d\\%d.png"%(ablation[i][0], ablation[i][1], ablation[i][2]), cv2.IMREAD_GRAYSCALE)

    img = np.where(height > 0, 0, img)

    plt.axis('off')
    plt.imshow(img, cmap=_cmap);

    plt.savefig('data/ablation_study/%s_%d_%d_%s/ground_truth.png'%(ablation[i][0], ablation[i][1], ablation[i][2], ablation[i][3]), bbox_inches='tight', pad_inches = 0)
    plt.close();

    saved_models = ['l1', 'l1_smooth', 'l2', 'ssim_l1', 'ssim_l1_smooth', 'ssim_sobel_plus_l1', 'ssim_with_sobel']
    loss_functions = [[l1_loss], [l1_smooth_loss], [l2_loss], [ssim_loss, l1_loss], [ssim_loss, l1_smooth_loss], [ssim_loss, l1_loss, sobel_loss], [ssim_loss, sobel_loss]]

    for j in range(len(saved_models)):
        tf.keras.backend.clear_session()
        down_stack, up_stack = get_generator_arch('resnet9', attn=False)
        deep_shadow = DeepShadow(512, 512, down_stack, up_stack, latitude=True, date=True, loss_funcs=loss_functions[j], type='resnet9', attention=False)
        deep_shadow.restore('training_checkpoints_new/uniform_cities_resnet_wo_attn__%s'%saved_models[j])

        height, pred = predict_shadow(deep_shadow.generator, 'data/heights_new/', ablation[i][0], ablation[i][3], 16, ablation[i][1], ablation[i][2], lat=True, dat=True)
        pred = np.where(height > 0, -1, pred)

        plt.axis('off')
        plt.imshow(pred, cmap=_cmap);

        plt.savefig('data/ablation_study/%s_%d_%d_%s/%s.png'%(ablation[i][0], ablation[i][1], ablation[i][2], ablation[i][3], saved_models[j]), bbox_inches='tight', pad_inches = 0)
        plt.close();
        test_on_image(deep_shadow.generator, 'data/heights_new/', 'data/shadows/', ablation[i][0], ablation[i][3], 16, ablation[i][1], ablation[i][2], lat=True, dat=True,
                      path = 'data/ablation_study/%s_%d_%d_%s/%s_metrics.png'%(ablation[i][0], ablation[i][1], ablation[i][2], ablation[i][3], saved_models[j]),
                    save=True)
    
    # *** #

    tf.keras.backend.clear_session()
    down_stack, up_stack = get_generator_arch('unet', attn=False)
    deep_shadow = DeepShadow(512, 512, down_stack, up_stack, latitude=True, date=True, loss_funcs=[l1_loss], type='unet', attention=False)
    deep_shadow.restore('training_checkpoints_new/uniform_cities_unet_wo_attn__l1')

    height, pred = predict_shadow(deep_shadow.generator, 'data/heights_new/', ablation[i][0], ablation[i][3], 16, ablation[i][1], ablation[i][2], lat=True, dat=True)
    pred = np.where(height > 0, -1, pred)
    
    plt.axis('off')
    plt.imshow(pred, cmap=_cmap);

    plt.savefig('data/ablation_study/%s_%d_%d_%s/unet_l1.png'%(ablation[i][0], ablation[i][1], ablation[i][2], ablation[i][3]), bbox_inches='tight', pad_inches = 0)
    plt.close();

    test_on_image(deep_shadow.generator, 'data/heights_new/', 'data/shadows/', ablation[i][0], ablation[i][3], 16, ablation[i][1], ablation[i][2], lat=True, dat=True,
                  path = 'data/ablation_study/%s_%d_%d_%s/unet_l1_metrics.png'%(ablation[i][0], ablation[i][1], ablation[i][2], ablation[i][3]),
                save=True)

  0%|          | 0/9 [00:00<?, ?it/s]

In [20]:
shadow_path = 'data/shadows/'
height_path = 'data/heights_new/'
files = glob.glob('tile_heights/*.csv')

def get_generator(_loss_functions, _saved_model, gen_arch='resnet9'):
    tf.keras.backend.clear_session()
    down_stack, up_stack = get_generator_arch(gen_arch, attn=False)
    deep_shadow = DeepShadow(512, 512, down_stack, up_stack, latitude=True, date=True, loss_funcs=_loss_functions, type=gen_arch, attention=False)
    deep_shadow.restore('training_checkpoints_new/%s'%_saved_model)

    return deep_shadow.generator

def get_rmse_list(tiles, gen, city, shadow_path, height_path):
    rmses = []
    for index, row in tqdm(tiles.iterrows(), total=tiles.shape[0], leave=False):
        # get i, j
        i = int(row['i'])
        j = int(row['j'])

        seasons = ['summer', 'winter', 'spring']
        rmse = 0
        for season in seasons:

            # generator, height_path, city, date, zoom, i, j, lat=True, dat=True
            height, pred = predict_shadow(gen, 'data/heights_new/', city, season, 16, i, j, lat=True, dat=True)
            pred = np.where(height > 0, -1, pred)

            gt = load_ground_truth(height, shadow_path, city, season, 16, i, j)
            gt = np.where(height > 0, -1, gt)

            pred = pred*0.5 + 0.5
            gt = gt*0.5 + 0.5

            rmse += np.sqrt(np.mean((pred - gt)**2))
                    
        rmse /= 3
        rmses.append(rmse)

    return rmses

In [23]:
test_cities = ['dc', 'nyc', 'joh', 'bue', 'bos', 'sea', 'tok']

for file in tqdm(files, total=len(files), leave=False):
    file = file.replace('\\','/')
    tiles = pd.read_csv(file)
    city = file.split('/')[-1].split('.')[0].split('-')[0]

    if city not in test_cities:
        continue

    # print(city)

    # saved_models = ['l2', 'ssim_l1', 'ssim_with_sobel']
    # loss_functions = [[l2_loss], [ssim_loss, l1_loss], [ssim_loss, sobel_loss]]


    # for k in tqdm(range(len(saved_models))):
    #     _saved_model = 'uniform_cities_resnet_wo_attn__%s'%saved_models[k]
    #     gen = get_generator(loss_functions[k], _saved_model, gen_arch='resnet9')

    #     rmses = get_rmse_list(tiles, gen, city, shadow_path, height_path)
        
    #     # print(rmses)
    #     tiles['%s'%saved_models[k]] = rmses
        
    #     tiles.to_csv(file, index=False)

    # gen = get_generator([l1_loss], 'uniform_cities_unet_wo_attn__l1', gen_arch='unet')
    # rmses = get_rmse_list(tiles, gen, city, shadow_path, height_path)
    # tiles['unet_l1'] = rmses
    # tiles.to_csv(file, index=False)

    # gen = get_generator([l1_loss], 'uniform_cities_resnet_wo_attn__l1', gen_arch='resnet9')
    # rmses = get_rmse_list(tiles, gen, city, shadow_path, height_path)
    # tiles['resnet_l1'] = rmses
    # tiles.to_csv(file, index=False)

    gen = get_generator([l1_loss], 'uniform_cities_resnet_wo_attn__ssim_sobel_plus_l1', gen_arch='resnet9')
    rmses = get_rmse_list(tiles, gen, city, shadow_path, height_path)
    tiles['resnet_ssim_sobel_l1'] = rmses
    tiles.to_csv(file, index=False)


  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/1340 [00:00<?, ?it/s]

  0%|          | 0/1037 [00:00<?, ?it/s]

  0%|          | 0/1536 [00:00<?, ?it/s]

  0%|          | 0/1990 [00:00<?, ?it/s]

  0%|          | 0/3742 [00:00<?, ?it/s]

  0%|          | 0/1203 [00:00<?, ?it/s]